# Dates pivot table

In [1]:
import json
from urllib.request import urlopen
import pandas as pd
import numpy as np

/home/ednilson/.virtualenvs/jupyter/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Get collection information from ArticleMeta 

In [2]:
AMC_URL = "http://articlemeta.scielo.org/api/v1/collection/identifiers/"
amc_data = pd.DataFrame(json.load(urlopen(AMC_URL)))

In [3]:
amc_data.head(6)

,acron,acron2,code,document_count,domain,has_analytics,is_active,journal_count,name,original_name,status,type
0,arg,ar,arg,38732.0,www.scielo.org.ar,True,True,"{'current': 125, 'deceased': 22}","{'pt': 'Argentina', 'en': 'Argentina', 'es': '...",Argentina,certified,journals
1,chl,cl,chl,63467.0,www.scielo.cl,True,True,"{'current': 105, 'deceased': 13, 'suspended': 1}","{'pt': 'Chile', 'en': 'Chile', 'es': 'Chile'}",Chile,certified,journals
2,col,co,col,69522.0,www.scielo.org.co,True,True,"{'current': 226, 'suspended': 7}","{'pt': 'Colombia', 'en': 'Colombia', 'es': 'Co...",Colombia,certified,journals
3,cub,cu,cub,33492.0,scielo.sld.cu,True,True,"{'current': 61, 'deceased': 2, 'suspended': 4}","{'pt': 'Cuba', 'en': 'Cuba', 'es': 'Cuba'}",Cuba,certified,journals
4,esp,es,esp,37815.0,scielo.isciii.es,True,True,"{'current': 43, 'deceased': 6, 'suspended': 11}","{'pt': 'Espanha', 'en': 'Spain', 'es': 'España'}",España,certified,journals
5,mex,mx,mex,63850.0,www.scielo.org.mx,True,True,"{'current': 159, 'deceased': 12, 'suspended': 44}","{'pt': 'Mexico', 'en': 'Mexico', 'es': 'Mexico'}",Mexico,certified,journals


Some collections won't be analyzed, mainly to avoid duplicates
(there are articles in more than one collection).
The `spa` (*Public Health* collection) should have part of it
kept in the result, but it's not a collection
whose journals/articles are assigned to a single country.
The collections below are linked to a single country:

In [4]:
dont_evaluate = ["bio", "cci", "cic", "ecu", "psi", "pry", "rve", "rvo", "rvt", "sss", "spa", "wid"]
amc_names_map = {
    "code": "collection",
    "acron2": "origin",
}
amc_pairs = amc_data \
    [(amc_data["acron2"].str.len() == 2) &
     ~amc_data["code"].isin(dont_evaluate)] \
    [list(amc_names_map.keys())] \
    .rename(columns=amc_names_map) \
    .assign(origin=lambda df: df["origin"].str.upper())
amc_pairs

,collection,origin
0,arg,AR
1,chl,CL
2,col,CO
3,cub,CU
4,esp,ES
5,mex,MX
6,prt,PT
8,scl,BR
11,sza,ZA
12,ven,VE


## ISSN selection from `spa`

These journals in the `spa` collection have the following countries:

In [5]:
spa_issn_country = pd.DataFrame([
    ("0021-2571", "IT"),
    ("0042-9686", "CH"),
    ("1020-4989", "US"),
    ("1555-7960", "US"),
], columns=["issn", "origin"])
spa_issn_country # For collection = "spa", only!

,issn,origin
0,0021-2571,IT
1,0042-9686,CH
2,1020-4989,US
3,1555-7960,US


## Languages dataset

This dataset is the
[Network spreadsheet/CSV pack](https://static.scielo.org/tabs/tabs_network.zip)
 which can be found in the
[SciELO Analytics report](https://analytics.scielo.org/w/reports)
web page.
The first two rows of it are:

#### Unzip the CSV file

In [14]:
import zipfile
# Use the Zip file in jcatalog/data/scielo
with zipfile.ZipFile('../../data/scielo/tabs_network_190128.zip', 'r') as zip_ref:
    zip_ref.extract('documents_dates.csv', 'csv_files')

In [15]:
dataset = pd.read_csv("csv_files/documents_dates.csv", keep_default_na=False)
dataset.head(3).T

/home/ednilson/.virtualenvs/jupyter/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2
extraction date,2019-01-10,2019-01-10,2019-01-10
study unit,document,document,document
collection,scl,scl,scl
ISSN SciELO,0100-879X,0100-879X,0100-879X
ISSN's,0100-879X;1414-431X,0100-879X;1414-431X,0100-879X;1414-431X
title at SciELO,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...
title thematic areas,Biological Sciences;Health Sciences,Biological Sciences;Health Sciences,Biological Sciences;Health Sciences
title is agricultural sciences,0,0,0
title is applied social sciences,0,0,0
title is biological sciences,1,1,1


#### Simplify the column names

In [10]:
names_map = {
    "ISSN SciELO": "issn",
    "title at SciELO":"title",
    "document publishing ID (PID SciELO)": "pid",
    "document type":"type",
    "document is citable": "is_citable",
    "document publishing year": "year"
}
df = dataset[list(names_map.keys())].rename(columns=names_map)
df[:5].T

,0,1,2,3,4
issn,0100-879X,0100-879X,0100-879X,0100-879X,0100-879X
title,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...,Brazilian Journal of Medical and Biological Re...
pid,S0100-879X1998000800006,S0100-879X1998000800011,S0100-879X1998000800005,S0100-879X1998000800009,S0100-879X1998000800010
type,research-article,rapid-communication,research-article,rapid-communication,rapid-communication
is_citable,1,1,1,1,1
year,1998,1998,1998,1998,1998


#### Add pub_year (ate_1996)

In [11]:
df["pub_year"] = np.where(df['year'] <= 1996, 'ate_1996', df["year"])

#### Add review type

In [12]:
df["tipo_review"] = np.where(df['type'] == "review-article", 1, 0)

#### Add citable_(language)

In [13]:
df["citable_pt"] = np.where((df['document_pt'] == 1) & (df['is_citable'] == 1), 1, 0)
df["citable_es"] = np.where((df['document_es'] == 1) & (df['is_citable'] == 1), 1, 0)
df["citable_en"] = np.where((df['document_en'] == 1) & (df['is_citable'] == 1), 1, 0)
df["citable_other_lang"] = np.where((df['document_other_languages'] == 1) & (df['is_citable'] == 1), 1, 0)

KeyError: 'document_pt'

In [12]:
df['sum_to_2_more_lang'] = np.sum([df['document_en'], df['document_pt'], df['document_es'], df['document_other_languages']], axis=0)
df[(df['sum_to_2_more_lang'] == 3)].T

,31760,32728,34448,36347,37268,37270,37272,39283,42191,42453,...,814642,814936,815029,815201,815701,815707,815824,816020,816085,816111
issn,0104-1169,0104-1169,0104-1169,0104-1169,0104-1169,0104-1169,0104-1169,0104-1169,0104-1169,0104-1169,...,0102-311X,0102-311X,0102-311X,0102-311X,0104-1290,0102-311X,0102-311X,0103-1104,1414-3283,0102-311X
title,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,Revista Latino-Americana de Enfermagem,...,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Saúde e Sociedade,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Saúde em Debate,"Interface - Comunicação, Saúde, Educação",Cadernos de Saúde Pública
pid,S0104-11692003000400001,S0104-11692003000600001,S0104-11692004000200001,S0104-11692003000500001,S0104-11692004000700001,S0104-11692004000700002,S0104-11692004000700003,S0104-11692004000400001,S0104-11692004000500001,S0104-11692004000600001,...,S0102-311X2018000700201,S0102-311X2018000800201,S0102-311X2018000700101,S0102-311X2018000900101,S0104-12902018000300682,S0102-311X2018001100101,S0102-311X2018001200201,S0103-11042018000400434,S1414-32832018000601757,S0102-311X2018001000101
type,editorial,editorial,editorial,editorial,editorial,research-article,research-article,editorial,editorial,editorial,...,editorial,editorial,editorial,editorial,research-article,editorial,editorial,research-article,undefined,editorial
languages,en;es;pt,en;es;pt,en;pt;es,en;pt;es,en;pt;es,en;pt;es,en;es;pt,en;es;pt,en;pt;es,en;es;pt,...,en;pt;es,en;pt;es,en;pt;es,en;es;pt,fr;en;pt,en;pt;es,en;pt;es,en;es;pt,en;pt;es,en;pt;es
is_citable,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
year,2003,2003,2004,2003,2004,2004,2004,2004,2004,2004,...,2018,2018,2018,2018,2018,2018,2018,2018,2018,2018
document_pt,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
document_es,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,1,1,1,1
document_en,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


#### Add 2 or more lang

In [13]:
df["doc_2_more_lang"] = np.where(((df['sum_to_2_more_lang']) >= 2), 1, 0)
df["citable_doc_2_more_lang"] = np.where((df['sum_to_2_more_lang'] >= 2) & (df['is_citable'] == 1), 1, 0)
# remove sum_to_2_more_lang column
del df['sum_to_2_more_lang']

df[(df["doc_2_more_lang"] == 1)].T

,11706,11710,11711,11713,11718,11721,11725,31273,31274,31275,...,844931,845966,845968,845970,845972,845974,845975,845976,845977,845978
issn,0102-311X,0102-311X,0102-311X,0102-311X,0102-311X,0102-311X,0102-311X,0066-782X,0066-782X,0066-782X,...,1688-9339,1688-9339,1688-9339,1688-9339,1688-9339,1688-9339,1688-9339,1688-9339,1688-9339,1688-9339
title,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Cadernos de Saúde Pública,Arquivos Brasileiros de Cardiologia,Arquivos Brasileiros de Cardiologia,Arquivos Brasileiros de Cardiologia,...,Odontoestomatología,Odontoestomatología,Odontoestomatología,Odontoestomatología,Odontoestomatología,Odontoestomatología,Odontoestomatología,Odontoestomatología,Odontoestomatología,Odontoestomatología
pid,S0102-311X1998000200011,S0102-311X1998000200015,S0102-311X1998000200016,S0102-311X1998000200018,S0102-311X1998000200024,S0102-311X1998000200003,S0102-311X1998000200007,S0066-782X2003001400001,S0066-782X2003001400004,S0066-782X2003001400005,...,S1688-93392015000100001,S1688-93392015000200003,S1688-93392015000200002,S1688-93392015000200006,S1688-93392015000200009,S1688-93392015000200007,S1688-93392015000200005,S1688-93392015000200001,S1688-93392015000200004,S1688-93392015000200008
type,research-article,research-article,research-article,brief-report,article-commentary,research-article,research-article,research-article,research-article,research-article,...,research-article,research-article,research-article,research-article,letter,case-report,research-article,editorial,research-article,case-report
languages,en;pt,es;pt,pt;es,en;pt,pt;es,en;pt,es;pt,en;pt,en;pt,en;pt,...,en;es,en;es,en;es,en;es,en;es,en;es,en;es,en;es,en;es,en;es
is_citable,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,1,0,1,1
year,1998,1998,1998,1998,1998,1998,1998,2003,2003,2003,...,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015
document_pt,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
document_es,0,1,1,0,1,0,1,0,0,0,...,1,1,1,1,1,1,1,1,1,1
document_en,1,0,0,1,0,1,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [14]:
df2 = df.rename(columns={"pid":"docs"})

In [15]:
df2.columns

Index(['issn', 'title', 'docs', 'type', 'languages', 'is_citable', 'year',
       'document_pt', 'document_es', 'document_en', 'document_other_languages',
       'pub_year', 'tipo_review', 'citable_pt', 'citable_es', 'citable_en',
       'citable_other_lang', 'doc_2_more_lang', 'citable_doc_2_more_lang'],
      dtype='object')

In [16]:
values_list = [
    "docs",
    "is_citable", 
    "tipo_review",
    "document_pt", 
    "document_es", 
    "document_en",
    "document_other_languages",
    "doc_2_more_lang",
    "citable_pt",
    "citable_es",
    "citable_en",
    "citable_other_lang",
    "citable_doc_2_more_lang"]

td = df2.pivot_table(
     index=["issn"],
     values=values_list,
     columns=["pub_year"],
     aggfunc=np.count_nonzero,
     fill_value=0)
td['docs'][:11].T

issn,0001-3714,0001-3765,0001-6002,0001-6365,0002-0591,0002-192X,0002-7014,0003-2573,0004-0592,0004-0614,0004-0622
pub_year,,,,,,,,,,,
1997,0,0,0,0,0,0,0,0,0,0,0
1998,33,0,0,0,0,0,0,0,0,0,0
1999,64,0,0,41,0,0,0,0,0,0,0
2000,0,119,31,50,0,0,0,0,0,0,61
2001,0,74,35,48,0,0,0,0,0,0,70
2002,0,104,31,60,0,18,0,0,0,0,54
2003,0,41,41,51,0,29,0,0,0,0,60
2004,0,85,48,49,0,27,0,34,0,0,64
2005,0,52,44,57,0,29,56,79,0,175,50


In [17]:
td.columns.levels

FrozenList([['citable_doc_2_more_lang', 'citable_en', 'citable_es', 'citable_other_lang', 'citable_pt', 'doc_2_more_lang', 'docs', 'document_en', 'document_es', 'document_other_languages', 'document_pt', 'is_citable', 'tipo_review'], ['1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', 'ate_1996']])

### Renames the labels for CSV

In [18]:
td.keys()
for k in td.keys():
    print(k)

('citable_doc_2_more_lang', '1997')
('citable_doc_2_more_lang', '1998')
('citable_doc_2_more_lang', '1999')
('citable_doc_2_more_lang', '2000')
('citable_doc_2_more_lang', '2001')
('citable_doc_2_more_lang', '2002')
('citable_doc_2_more_lang', '2003')
('citable_doc_2_more_lang', '2004')
('citable_doc_2_more_lang', '2005')
('citable_doc_2_more_lang', '2006')
('citable_doc_2_more_lang', '2007')
('citable_doc_2_more_lang', '2008')
('citable_doc_2_more_lang', '2009')
('citable_doc_2_more_lang', '2010')
('citable_doc_2_more_lang', '2011')
('citable_doc_2_more_lang', '2012')
('citable_doc_2_more_lang', '2013')
('citable_doc_2_more_lang', '2014')
('citable_doc_2_more_lang', '2015')
('citable_doc_2_more_lang', '2016')
('citable_doc_2_more_lang', '2017')
('citable_doc_2_more_lang', '2018')
('citable_doc_2_more_lang', '2019')
('citable_doc_2_more_lang', 'ate_1996')
('citable_en', '1997')
('citable_en', '1998')
('citable_en', '1999')
('citable_en', '2000')
('citable_en', '2001')
('citable_en', '2

In [19]:
newlabel = []
for k in td.keys():
    newlabel.append(k[0]+'_'+k[1])

In [20]:
newlabel

['citable_doc_2_more_lang_1997',
 'citable_doc_2_more_lang_1998',
 'citable_doc_2_more_lang_1999',
 'citable_doc_2_more_lang_2000',
 'citable_doc_2_more_lang_2001',
 'citable_doc_2_more_lang_2002',
 'citable_doc_2_more_lang_2003',
 'citable_doc_2_more_lang_2004',
 'citable_doc_2_more_lang_2005',
 'citable_doc_2_more_lang_2006',
 'citable_doc_2_more_lang_2007',
 'citable_doc_2_more_lang_2008',
 'citable_doc_2_more_lang_2009',
 'citable_doc_2_more_lang_2010',
 'citable_doc_2_more_lang_2011',
 'citable_doc_2_more_lang_2012',
 'citable_doc_2_more_lang_2013',
 'citable_doc_2_more_lang_2014',
 'citable_doc_2_more_lang_2015',
 'citable_doc_2_more_lang_2016',
 'citable_doc_2_more_lang_2017',
 'citable_doc_2_more_lang_2018',
 'citable_doc_2_more_lang_2019',
 'citable_doc_2_more_lang_ate_1996',
 'citable_en_1997',
 'citable_en_1998',
 'citable_en_1999',
 'citable_en_2000',
 'citable_en_2001',
 'citable_en_2002',
 'citable_en_2003',
 'citable_en_2004',
 'citable_en_2005',
 'citable_en_2006',
 'ci

In [21]:
newlabel[::24]

['citable_doc_2_more_lang_1997',
 'citable_en_1997',
 'citable_es_1997',
 'citable_other_lang_1997',
 'citable_pt_1997',
 'doc_2_more_lang_1997',
 'docs_1997',
 'document_en_1997',
 'document_es_1997',
 'document_other_languages_1997',
 'document_pt_1997',
 'is_citable_1997',
 'tipo_review_1997']

In [22]:
td.columns = newlabel

In [23]:
td.T

issn,0001-3714,0001-3765,0001-6002,0001-6365,0002-0591,0002-192X,0002-7014,0003-2573,0004-0592,0004-0614,...,2518-4431,2520-9868,2526-8910,2531-0488,2531-1379,2545-7756,2594-1321,2594-9985,2595-3192,2619-6573
citable_doc_2_more_lang_1997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
citable_doc_2_more_lang_1998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
citable_doc_2_more_lang_1999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
citable_doc_2_more_lang_2000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
citable_doc_2_more_lang_2001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
citable_doc_2_more_lang_2002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
citable_doc_2_more_lang_2003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
citable_doc_2_more_lang_2004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
citable_doc_2_more_lang_2005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
citable_doc_2_more_lang_2006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
td.to_csv("output/td_documents_dates_network.csv")